In [36]:
import csv 
import os 
from zipcodes import pullzipcodes

####import list of NYC only zipcodes to filter out RefUSA data

In [37]:
directory = os.environ['data_path']
files = sorted(os.listdir(directory))
df10 = directory + "/" + files[0]

In [38]:
files

['Hist_Bus_2010.csv',
 'Hist_Bus_2011.csv',
 'Hist_Bus_2012.csv',
 'Hist_Bus_2013.csv',
 'Hist_Bus_2014.csv']

In [39]:
zips = pullzipcodes()

####read RefUSA data and filter by zipcodes 

In [40]:
import pandas as pd

In [41]:
ref10 = pd.read_csv(df10, dtype = {'ZIP': object, 'CENSUS_TRACT_2010': object, 'CENSUS_BG_2010': object})

In [42]:
ref10.b = ref10[ref10['ZIP'].isin(zips)]

In [30]:
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['CB2010', 'C', 4, 0],
 ['BoroCode', 'C', 1, 0],
 ['BoroName', 'C', 32, 0],
 ['CT2010', 'C', 6, 0],
 ['BCTCB2010', 'C', 11, 0],
 ['Shape_Leng', 'F', 19, 11],
 ['Shape_Area', 'F', 19, 11]]

In [34]:
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['CTLabel', 'C', 7, 0],
 ['BoroCode', 'C', 1, 0],
 ['BoroName', 'C', 32, 0],
 ['CT2010', 'C', 6, 0],
 ['BoroCT2010', 'C', 7, 0],
 ['CDEligibil', 'C', 1, 0],
 ['NTACode', 'C', 4, 0],
 ['NTAName', 'C', 75, 0],
 ['PUMA', 'C', 4, 0],
 ['Shape_Leng', 'F', 19, 11],
 ['Shape_Area', 'F', 19, 11]]

In [35]:
sf.record(494)

['326',
 '2',
 'Bronx',
 '032600',
 '2032600',
 'I',
 'BX31',
 'Allerton-Pelham Gardens',
 '3704',
 '8.80039965477e+003',
 '3.56017830342e+006']

####read Borough Boundary Files from NYC Pluto 

In [43]:
import shapefile
from pyproj import Proj, transform 
from rtree import index 

In [46]:
#sf = shapefile.Reader("nybb_15b/nybb.shp")
sf = shapefile.Reader("nycb2010_15b/nycb2010.shp")
shapes = sf.shapes()

In [45]:
sf.record(494) 
#information - cb, boro, boro_name, cen_tract, boro+cb+ct, .. , ..

['326',
 '2',
 'Bronx',
 '032600',
 '2032600',
 'I',
 'BX31',
 'Allerton-Pelham Gardens',
 '3704',
 '8.80039965477e+003',
 '3.56017830342e+006']

In [11]:
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['CB2010', 'C', 4, 0],
 ['BoroCode', 'C', 1, 0],
 ['BoroName', 'C', 32, 0],
 ['CT2010', 'C', 6, 0],
 ['BCTCB2010', 'C', 11, 0],
 ['Shape_Leng', 'F', 19, 11],
 ['Shape_Area', 'F', 19, 11]]

In [47]:
def convert(li):
    f = 0.304800
    inProj = Proj(init='epsg:2263')
    outProj = Proj(init='epsg:4326')
    x0,y0 = li[0]*f, li[1]*f
    x1,y1 = li[2]*f, li[3]*f
    
    x2,y2 = transform(inProj,outProj,x0,y0)
    x3,y3 = transform(inProj,outProj,x1,y1)
    return (x2,y2,x3,y3)

In [48]:
convert(shapes[0].bbox) # bbox provides bottom left and top right corners of any polygon 

(-74.0823760429353, 40.64297065408437, -74.07914370756991, 40.64479304833291)

####Build rtree

In [49]:
idx = index.Index()

In [50]:
loc = 0 
for i in shapes:
    corners = shapes[loc].bbox 
    left, bottom, right, top = convert(corners)
    idx.insert(loc, (left, bottom, right, top))
    loc += 1

In [51]:
def most_common(lst):
    return max(lst, key=lst.count)

####This function extracts the borough of a business

In [52]:
def findBorough(row):
    a = row["LONGITUDE_2010"]
    b = row["LATITUDE_2010"]
    
    if pd.isnull(a) == True: 
        return 'NA' 
    
    else: 
        li = list(idx.intersection((a, b)))
        if len(li) != 0:
            li2 = [sf.record(ix)[2] for ix in li]
            bor = most_common(li2)
            return bor
        else: 
            return 'NA'    

####This function extracts the census tract of a business

In [53]:
def findCT(row):
    a = row["LONGITUDE_2010"]
    b = row["LATITUDE_2010"]
    
    if pd.isnull(a) == True: 
        return 'NA' 
    else:
        li = list(idx.intersection((a, b)))
        
        if len(li) != 0:
            li2 = [sf.record(ix)[3] for ix in li]
            ct = most_common(li2)
            return ct

####This function extracts the census block of a business.  There are several values; we take the census block that occurs most in any list

In [54]:
def findCB(row):
    a = row["LONGITUDE_2010"]
    b = row["LATITUDE_2010"]
    
    if pd.isnull(a) == True: 
        return 'NA' 
    else:
        li = list(idx.intersection((a, b)))
        
        if len(li) != 0:
            li2 = [sf.record(ix)[0][0] for ix in li]
            ct = most_common(li2)
            return ct

In [55]:
pd.set_option('chained_assignment','warn')

In [23]:
ref10.b['SHP_BOROUGH'] = ref10.b.apply(findBorough, axis=1)

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
ref10.b['SHP_CENSUS_TRACT'] = ref10.b.apply(findCT, axis=1)

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [25]:
ref10.b['SHP_CENSUS_BLOCK'] = ref10.b.apply(findCB, axis=1)

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [39]:
# testdf = ref10.b[['CENSUS_TRACT_2010', 'CENSUS_BG_2010', 'SHP_CENSUS_TRACT', 'SHP_CENSUS_BLOCK']]


In [40]:
# testdf['check'] = testdf['CENSUS_BG_2010'] == testdf['SHP_CENSUS_BLOCK']

In [41]:
# testdf['check'].value_counts()

####read nys_acs data (contains New York, Richmond, Kings, Queens, and Bronx Counties) 

In [57]:
acs = pd.read_csv("nys_mi.csv", dtype = {'Geo_TRACT': object, 'Geo_BLKGRP': object, 'Geo_COUNTY': object})

In [58]:
counties = ['005', '047', '061', '081', '085']

counties_dict = {'005': 'Bronx',
                 '047': 'Brooklyn',
                 '061': 'Manhattan',
                 '081': 'Queens',
                 '085': 'Staten Island'}
 
acs.b = acs[acs['Geo_COUNTY'].isin(counties)][['Geo_NAME', 'Geo_COUNTY', 'Geo_TRACT', 'Geo_BLKGRP', 'SE_T057_001']]
acs.b = acs.b.rename(columns = {'Geo_TRACT': 'SHP_CENSUS_TRACT', 'Geo_BLKGRP': 'SHP_CENSUS_BLOCK'})

In [44]:
def labelBorough(row):
    boro = counties_dict[row['Geo_COUNTY']]
    return boro

In [59]:
len(acs)

15463

In [45]:
acs.b['SHP_BOROUGH'] = acs.b.apply(labelBorough, axis = 1)

In [80]:
acs.b.head()

,Geo_NAME,Geo_COUNTY,SHP_CENSUS_TRACT,SHP_CENSUS_BLOCK,SE_T057_001
279,"Block Group 0, Census Tract 1, Bronx County, N...",005,000100,0,NaN
280,"Block Group 1, Census Tract 1, Bronx County, N...",005,000100,1,NaN
281,"Block Group 0, Census Tract 2, Bronx County, N...",005,000200,0,NaN
282,"Block Group 1, Census Tract 2, Bronx County, N...",005,000200,1,61779
283,"Block Group 2, Census Tract 2, Bronx County, N...",005,000200,2,70603


####Merge Files 

In [46]:
result = pd.merge(ref10.b, acs.b, how = 'inner', on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT','SHP_CENSUS_BLOCK'])

####Append the LMI attribute 

##### The suggested threshold for LMI is a household income of $55,113

In [48]:
LMI_threshold = 55113 

In [49]:
def LMI(row):
    return row['SE_T057_001'] < LMI_threshold

In [50]:
result['LMI'] = result.apply(LMI, axis = 1)

In [51]:
result.to_csv("Ref14_LMI.csv")

In [97]:
test = [] 
for i, j in enumerate(range(len(shapes))):
    var = sf.record(i)[4][:-3]
    test.append(var)

In [99]:
len(set(test))

6373

In [22]:
ls Ref_LMI

grouped_files/      Ref10_LMI.csv  Ref12_LMI.csv  Ref14_LMI.csv
GroupedFiles.ipynb  Ref11_LMI.csv  Ref13_LMI.csv


In [26]:
test = pd.read_csv("Ref_LMI/Ref10_LMI.csv", index_col = 0)

In [28]:
test.ix[1]

CONAME                                             TRITON SHIPPING INC
ADDR                                                      1041 3RD AVE
CITY                                                          NEW YORK
STATE                                                               NY
ZIP                                                              10065
INDFRM                                                               2
PRMSIC                                                          448903
PRMSICD                                            STEAMSHIP COMPANIES
PNACODE                                                   4.872101e+07
PNATITL                      SCENIC & SIGHTSEEING TRANSPORTATION WATER
EMPSDT                                                               4
SLSVDT                                                             424
HDBRCH                                                             NaN
ABI                                                            6680995
SUBNUM